In [1]:
import pandas as pd
#import gspread
import requests 
import io
import matplotlib.pyplot as plt
import scipy
import numpy as np

#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

In [2]:
# AL Train Data Set
ALdata = pd.read_csv('./AL/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
ALdata.head(15)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [3]:
# CN Train Data Set
CNdata = pd.read_csv('./CN/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
CNdata.head(15)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,无忌,B-neutral
7,哥哥,I-neutral
8,对,O
9,敏敏,B-neutral


In [4]:
# EN Train Data Set
ENdata = pd.read_csv('./EN/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
ENdata.head(15)

,Word,Tag
0,Municipal,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,safer,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [5]:
# SG Train Data Set
SGdata = pd.read_csv('./SG/train', low_memory=False, lineterminator ='\n', delimiter = ' ', header = None, names = ['Word', 'Tag'])
SGdata.head(15)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


In [6]:
print('EN:')
print(ENdata['Tag'].value_counts())
print('\n')

print('AL:')
print(ALdata['Tag'].value_counts())
print('\n')

print('CN:')
print(CNdata['Tag'].value_counts())
print('\n')

print('SG:')
print(SGdata['Tag'].value_counts())
print('\n')

EN:
I-NP       54591
B-NP       47305
O          23872
B-PP       18387
B-VP       18261
I-VP       10159
B-ADVP      3565
B-SBAR      1899
B-ADJP      1751
I-ADJP       574
B-PRT        468
I-ADVP       363
I-PP         223
I-CONJP       64
B-CONJP       49
I-SBAR        48
B-INTJ        26
B-LST         11
I-INTJ         7
I-UCP          4
B-UCP          1
Name: Tag, dtype: int64


AL:
I-POI          28472
I-ROAD         15178
I-DISTRICT     13364
I-TOWN         10748
I-CITY         10430
I-PROV          8666
B-POI           7348
B-DISTRICT      6856
B-ROAD          6324
B-CITY          5697
B-ROADNO        5061
I-ROADNO        4666
B-TOWN          4612
I-SUBPOI        4557
B-PROV          4488
B-REDUNDANT     4137
I-REDUNDANT     3668
B-HOUSENO       3455
I-COMMUNITY     3148
B-ROOMNO        3107
I-HOUSENO       3045
I-PERSON        2232
I-DEVZONE       1834
B-SUBPOI        1650
I-CELLNO        1488
B-COMMUNITY     1479
B-CELLNO        1326
I-FLOORNO       1237
B-FLOORNO       1229


In [7]:
# Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation)
# e(x|y) = Count (y -> x)/Count(y)
 
def emissionEstimate(data, word, tag):
    tagcheck = (data.Tag == tag)
    y = tagcheck.sum()
    xy = (data.Word[tagcheck] == word).sum()
    
    return xy/y

In [8]:
# replace the words that appear less than k times in the training set with
# special word token #UNK# before training. This leads to a "modified training set"
# if a word does not appear in the "modified training set", we replace that with #UNK# as well
# change k to whatever value you want

#Smoothing of EN
#k = 3
EN = pd.DataFrame(ENdata['Word'].value_counts()< 3)
EN.apply(lambda val: val == True)
k1 = EN.index[EN['Word'] == True].tolist()
ENdata.loc[ENdata['Word'].isin(k1), 'Word'] = '#UNK#'
ENdata.head(20)

,Word,Tag
0,#UNK#,B-NP
1,bonds,I-NP
2,are,B-VP
3,generally,B-ADVP
4,a,B-ADJP
5,bit,I-ADJP
6,#UNK#,I-ADJP
7,than,B-PP
8,corporate,B-NP
9,bonds,I-NP


In [9]:
#Smoothing of AL
AL = pd.DataFrame(ALdata['Word'].value_counts()<3)
AL.apply(lambda val: val == True)
k2 = AL.index[AL['Word'] == True].tolist()
ALdata.loc[ALdata['Word'].isin(k2), 'Word'] = '#UNK#'
ALdata.head(20)

,Word,Tag
0,小,B-DISTRICT
1,港,I-DISTRICT
2,四,B-POI
3,方,I-POI
4,家,I-POI
5,园,I-POI
6,17,B-HOUSENO
7,幢,I-HOUSENO
8,1363,B-CELLNO
9,室,I-CELLNO


In [10]:
#Smoothing of CN
CN = pd.DataFrame(CNdata['Word'].value_counts()<3)
CN.apply(lambda val: val == True)
k3 = CN.index[CN['Word'] == True].tolist()
CNdata.loc[CNdata['Word'].isin(k3), 'Word'] = '#UNK#'
CNdata.head(20)

,Word,Tag
0,这,O
1,大概,O
2,是,O
3,所有,O
4,版本,O
5,里,O
6,#UNK#,B-neutral
7,哥哥,I-neutral
8,对,O
9,#UNK#,B-neutral


In [11]:
#Smoothing of SG
SG = pd.DataFrame(SGdata['Word'].value_counts()<3)
SG.apply(lambda val: val == True)
k4 = SG.index[SG['Word'] == True].tolist()
SGdata.loc[SGdata['Word'].isin(k4), 'Word'] = '#UNK#'
SGdata.head(20)

,Word,Tag
0,I'm,O
1,about,O
2,to,O
3,see,O
4,Adam,B-positive
5,Levine,I-positive
6,and,O
7,I'm,O
8,probably,O
9,going,O


Goal: Obtain a dataset with Word, Tag, Emission

Steps
1) Obtain count of tags per word
2) Find the Count of the different tags across the whole dataset
3) For each tag for each word, find out what the total count for each tag
4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
5) Keep only the word/tag with highest emission value

Finally, export pdf

In [12]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#EN

# 1)
ENWordTagcount = pd.DataFrame({'Count' : ENdata.groupby(['Word', 'Tag']).size()}).reset_index()
ENWordTagcount

,Word,Tag,Count
0,!,O,15
1,#,B-ADJP,3
2,#,B-NP,20
3,#,I-NP,10
4,#,O,1
5,#UNK#,B-ADJP,272
6,#UNK#,B-ADVP,222
7,#UNK#,B-INTJ,9
8,#UNK#,B-LST,1
9,#UNK#,B-NP,3903


In [13]:
# 2)
ENTagcount = pd.DataFrame({'Global Count' :ENdata['Tag'].value_counts()}).reset_index()
ENTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

ENTagcount

,Tag,Global Count
0,I-NP,54591
1,B-NP,47305
2,O,23872
3,B-PP,18387
4,B-VP,18261
5,I-VP,10159
6,B-ADVP,3565
7,B-SBAR,1899
8,B-ADJP,1751
9,I-ADJP,574


In [14]:
# 3
ENTotalTagCount = pd.merge(ENWordTagcount, ENTagcount, on='Tag')
ENTotalTagCount

,Word,Tag,Count,Global Count
0,!,O,15,23872
1,#,O,1,23872
2,#UNK#,O,56,23872
3,$,O,2,23872
4,&,O,3,23872
5,',O,19,23872
6,'',O,1168,23872
7,'s,O,2,23872
8,",",O,8705,23872
9,-,O,11,23872


In [15]:
# 4)
ENTotalTagCount['Emission'] = ENTotalTagCount['Count']/ENTotalTagCount['Global Count']
ENTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,!,O,15,23872,0.000628
1,#,O,1,23872,0.000042
2,#UNK#,O,56,23872,0.002346
3,$,O,2,23872,0.000084
4,&,O,3,23872,0.000126
5,',O,19,23872,0.000796
6,'',O,1168,23872,0.048928
7,'s,O,2,23872,0.000084
8,",",O,8705,23872,0.364653
9,-,O,11,23872,0.000461


In [16]:
# 5)
idx = ENTotalTagCount.groupby(['Word'])['Emission'].transform(max) == ENTotalTagCount['Emission']
ENmodel = ENTotalTagCount[idx]

In [17]:
#open file

#export file
r = open("./EN/dev.in","r",encoding="utf8")
f = open("./EN/dev.p2.out","w",encoding="UTF-8")

ls = []
for line in r.readlines():
    if (line !='\n'and line.strip('\n') in ENmodel['Word']) : #check if word not newline and exist in model
        f.write(line.strip('\n')+ " " + ENmodel['Tag'].loc[ENmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'): #check if word does not exist in model
        f.write(line.strip('\n')+ " " + ENmodel['Tag'].loc[ENmodel['Word']== "#UNK#"].item() + "\n" )
    else: #check if word is space
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


In [18]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#AL

# 1)
ALWordTagcount = pd.DataFrame({'Count' : ALdata.groupby(['Word', 'Tag']).size()}).reset_index()
ALWordTagcount

,Word,Tag,Count
0,#UNK#,B-ASSIST,4
1,#UNK#,B-CELLNO,5
2,#UNK#,B-CITY,18
3,#UNK#,B-COMMUNITY,47
4,#UNK#,B-DEVZONE,2
5,#UNK#,B-DISTRICT,38
6,#UNK#,B-FLOORNO,2
7,#UNK#,B-HOUSENO,40
8,#UNK#,B-PERSON,54
9,#UNK#,B-POI,145


In [19]:
# 2)
ALTagcount = pd.DataFrame({'Global Count' :ALdata['Tag'].value_counts()}).reset_index()
ALTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

ALTagcount

,Tag,Global Count
0,I-POI,28472
1,I-ROAD,15178
2,I-DISTRICT,13364
3,I-TOWN,10748
4,I-CITY,10430
5,I-PROV,8666
6,B-POI,7348
7,B-DISTRICT,6856
8,B-ROAD,6324
9,B-CITY,5697


In [20]:
# 3
ALTotalTagCount = pd.merge(ALWordTagcount, ALTagcount, on='Tag')
ALTotalTagCount

,Word,Tag,Count,Global Count
0,#UNK#,B-ASSIST,4,801
1,-,B-ASSIST,2,801
2,1,B-ASSIST,1,801
3,150,B-ASSIST,1,801
4,2,B-ASSIST,3,801
5,200,B-ASSIST,1,801
6,400,B-ASSIST,1,801
7,5,B-ASSIST,1,801
8,A,B-ASSIST,4,801
9,B,B-ASSIST,2,801


In [21]:
# 4)
ALTotalTagCount['Emission'] = ALTotalTagCount['Count']/ALTotalTagCount['Global Count']
ALTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,#UNK#,B-ASSIST,4,801,0.004994
1,-,B-ASSIST,2,801,0.002497
2,1,B-ASSIST,1,801,0.001248
3,150,B-ASSIST,1,801,0.001248
4,2,B-ASSIST,3,801,0.003745
5,200,B-ASSIST,1,801,0.001248
6,400,B-ASSIST,1,801,0.001248
7,5,B-ASSIST,1,801,0.001248
8,A,B-ASSIST,4,801,0.004994
9,B,B-ASSIST,2,801,0.002497


In [22]:
# 5)
idx = ALTotalTagCount.groupby(['Word'])['Emission'].transform(max) == ALTotalTagCount['Emission']
ALmodel = ALTotalTagCount[idx]
ALmodel

,Word,Tag,Count,Global Count,Emission
6,400,B-ASSIST,1,801,0.001248
11,d,B-ASSIST,1,801,0.001248
18,与,B-ASSIST,79,801,0.098627
19,东,B-ASSIST,44,801,0.054931
21,之,B-ASSIST,2,801,0.002497
23,交,B-ASSIST,90,801,0.112360
24,以,B-ASSIST,10,801,0.012484
27,内,B-ASSIST,32,801,0.039950
28,出,B-ASSIST,2,801,0.002497
29,到,B-ASSIST,2,801,0.002497


In [23]:
ALmodel['Tag'].loc[ALmodel['Word']=='400'].item()

'B-ASSIST'

In [24]:
#open file
r = open("./AL/dev.in","r",encoding="UTF-8")
f = open("./AL/dev.p2.out","w",encoding="UTF-8")

#export file

for line in r.readlines():
    if (line !='\n'and line.strip('\n') in ALmodel['Word']) : 
        f.write(line.strip('\n')+ " " + ALmodel['Tag'].loc[ALmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'):
        f.write(line.strip('\n')+ " " + ALmodel['Tag'].loc[ALmodel['Word']== "#UNK#"].item() + "\n" )
    else:
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


In [25]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#CN

# 1)
CNWordTagcount = pd.DataFrame({'Count' : CNdata.groupby(['Word', 'Tag']).size()}).reset_index()
CNWordTagcount

,Word,Tag,Count
0,!,I-neutral,4
1,!,I-positive,4
2,!,O,68
3,#,B-negative,11
4,#,B-neutral,453
5,#,B-positive,401
6,#,I-neutral,5
7,#,I-positive,5
8,#,O,2043
9,##,B-negative,2


In [26]:
# 2)
CNTagcount = pd.DataFrame({'Global Count' :CNdata['Tag'].value_counts()}).reset_index()
CNTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

CNTagcount

,Tag,Global Count
0,O,123304
1,I-neutral,6030
2,B-neutral,5252
3,I-positive,3845
4,B-positive,3087
5,B-negative,449
6,I-negative,275


In [27]:
# 3
CNTotalTagCount = pd.merge(CNWordTagcount, CNTagcount, on='Tag')
CNTotalTagCount

,Word,Tag,Count,Global Count
0,!,I-neutral,4,6030
1,#,I-neutral,5,6030
2,#UNK#,I-neutral,1501,6030
3,&,I-neutral,4,6030
4,',I-neutral,7,6030
5,(,I-neutral,12,6030
6,),I-neutral,11,6030
7,+,I-neutral,2,6030
8,",",I-neutral,6,6030
9,-,I-neutral,28,6030


In [28]:
# 4)
CNTotalTagCount['Emission'] = CNTotalTagCount['Count']/CNTotalTagCount['Global Count']
CNTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,!,I-neutral,4,6030,0.000663
1,#,I-neutral,5,6030,0.000829
2,#UNK#,I-neutral,1501,6030,0.248922
3,&,I-neutral,4,6030,0.000663
4,',I-neutral,7,6030,0.001161
5,(,I-neutral,12,6030,0.001990
6,),I-neutral,11,6030,0.001824
7,+,I-neutral,2,6030,0.000332
8,",",I-neutral,6,6030,0.000995
9,-,I-neutral,28,6030,0.004643


In [29]:
# 5)
idx = CNTotalTagCount.groupby(['Word'])['Emission'].transform(max) == CNTotalTagCount['Emission']
CNmodel = CNTotalTagCount[idx]
CNmodel

,Word,Tag,Count,Global Count,Emission
3,&,I-neutral,4,6030,0.000663
4,',I-neutral,7,6030,0.001161
5,(,I-neutral,12,6030,0.001990
6,),I-neutral,11,6030,0.001824
12,0,I-neutral,2,6030,0.000332
16,1021,I-neutral,1,6030,0.000166
18,12,I-neutral,4,6030,0.000663
20,150,I-neutral,1,6030,0.000166
21,16,I-neutral,2,6030,0.000332
27,3000,I-neutral,1,6030,0.000166


In [30]:
#open file
#export file 

r = open("./CN/dev.in","r",encoding="utf8")
f = open("./CN/dev.p2.out","w",encoding="UTF-8")

ls = []
for line in r.readlines():
    if (line !='\n'and line.strip('\n') in CNmodel['Word']) : 
        f.write(line.strip('\n')+ " " + CNmodel['Tag'].loc[CNmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'):
        f.write(line.strip('\n')+ " " + CNmodel['Tag'].loc[CNmodel['Word']== "#UNK#"].item() + "\n" )
    else:
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


In [31]:
# Goal: Obtain a dataset with Word, Tag, Emission

# Steps
# 1) Obtain count of tags per word
# 2) Find the Count of the different tags across the whole dataset
# 3) For each tag for each word, find out what the total count for each tag
# 4) Obtain emission count with Word, Tag, Emission by dividing step 1 and 4
# 5) Keep only the word/tag with highest emission value

#SG

# 1)
SGWordTagcount = pd.DataFrame({'Count' : SGdata.groupby(['Word', 'Tag']).size()}).reset_index()
SGWordTagcount

,Word,Tag,Count
0,O\n,O,23
1,!,I-negative,3
2,!,I-neutral,2
3,!,I-positive,2
4,!,O,2469
5,#,O,20
6,#Accounting,O,6
7,#Albany,B-neutral,2
8,#Albany,O,3
9,#America,B-neutral,1


In [32]:
# 2)
SGTagcount = pd.DataFrame({'Global Count' :SGdata['Tag'].value_counts()}).reset_index()
SGTagcount.rename(columns = {'index' : 'Tag'}, inplace= True)

SGTagcount

,Tag,Global Count
0,O,133993
1,B-neutral,9736
2,I-neutral,9050
3,B-positive,3521
4,I-positive,2309
5,B-negative,1818
6,I-negative,708


In [33]:
# 3
SGTotalTagCount = pd.merge(SGWordTagcount, SGTagcount, on='Tag')
SGTotalTagCount

,Word,Tag,Count,Global Count
0,O\n,O,23,133993
1,!,O,2469,133993
2,#,O,20,133993
3,#Accounting,O,6,133993
4,#Albany,O,3,133993
5,#America,O,2,133993
6,#Art,O,4,133993
7,#Aurora,O,1,133993
8,#Automotive,O,6,133993
9,#BBOTT,O,3,133993


In [34]:
# 4)
SGTotalTagCount['Emission'] = SGTotalTagCount['Count']/SGTotalTagCount['Global Count']
SGTotalTagCount

,Word,Tag,Count,Global Count,Emission
0,O\n,O,23,133993,0.000172
1,!,O,2469,133993,0.018426
2,#,O,20,133993,0.000149
3,#Accounting,O,6,133993,0.000045
4,#Albany,O,3,133993,0.000022
5,#America,O,2,133993,0.000015
6,#Art,O,4,133993,0.000030
7,#Aurora,O,1,133993,0.000007
8,#Automotive,O,6,133993,0.000045
9,#BBOTT,O,3,133993,0.000022


In [35]:
# 5)
idx = SGTotalTagCount.groupby(['Word'])['Emission'].transform(max) == SGTotalTagCount['Emission']
SGmodel = SGTotalTagCount[idx]
SGmodel

,Word,Tag,Count,Global Count,Emission
0,O\n,O,23,133993,0.000172
1,!,O,2469,133993,0.018426
2,#,O,20,133993,0.000149
3,#Accounting,O,6,133993,0.000045
6,#Art,O,4,133993,0.000030
8,#Automotive,O,6,133993,0.000045
9,#BBOTT,O,3,133993,0.000022
10,#BYZ,O,3,133993,0.000022
11,#BadThingsOniTunes,O,4,133993,0.000030
12,#Banking,O,5,133993,0.000037


In [36]:
#open file
#export file
r = open("./SG/dev.in","r",encoding="utf8")
f = open("./SG/dev.p2.out","w",encoding="UTF-8")

ls = []
for line in r.readlines():
    if (line !='\n'and line.strip('\n') in SGmodel['Word']) : 
        f.write(line.strip('\n')+ " " + SGmodel['Tag'].loc[SGmodel['Word']== line.strip('\n')].item() + "\n" )
    elif (line !='\n'):
        f.write(line.strip('\n')+ " " + SGmodel['Tag'].loc[SGmodel['Word']== "#UNK#"].item() + "\n" )
    else:
        f.write(line)
f.close()
r.close()
    
    #ls.append(line.strip('\n'))
#print(r.readline())
print(ls)

[]


Evaluation Scripts

In [37]:
!py  ./EvalScript/evalResult.py ./EN/dev.out ./EN/dev.p2.out


#Entity in gold data: 13179
#Entity in prediction: 26131

#Correct Entity : 7310
Entity  precision: 0.2797
Entity  recall: 0.5547
Entity  F: 0.3719

#Correct Sentiment : 1
Sentiment  precision: 0.0000
Sentiment  recall: 0.0001
Sentiment  F: 0.0001


In [38]:
!py   ./EvalScript/evalResult.py ./AL/dev.out ./AL/dev.p2.out


#Entity in gold data: 8408
#Entity in prediction: 24663

#Correct Entity : 926
Entity  precision: 0.0375
Entity  recall: 0.1101
Entity  F: 0.0560

#Correct Sentiment : 282
Sentiment  precision: 0.0114
Sentiment  recall: 0.0335
Sentiment  F: 0.0171


In [39]:
!py  ./EvalScript/evalResult.py ./CN/dev.out ./CN/dev.p2.out


#Entity in gold data: 1478
#Entity in prediction: 23531

#Correct Entity : 700
Entity  precision: 0.0297
Entity  recall: 0.4736
Entity  F: 0.0560

#Correct Sentiment : 63
Sentiment  precision: 0.0027
Sentiment  recall: 0.0426
Sentiment  F: 0.0050


In [40]:
!py ./EvalScript/evalResult.py ./SG/dev.out ./SG/dev.p2.out


#Entity in gold data: 4537
#Entity in prediction: 47656

#Correct Entity : 2483
Entity  precision: 0.0521
Entity  recall: 0.5473
Entity  F: 0.0951

#Correct Sentiment : 572
Sentiment  precision: 0.0120
Sentiment  recall: 0.1261
Sentiment  F: 0.0219
